In [148]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import glob

print(tf.__version__)

batch_size = 32
img_height = 180
img_width = 180

2.7.0


In [149]:
# data_dir = 'dataset_lettuce'
#
# train_ds = tf.keras.utils.image_dataset_from_directory(
#   data_dir,
#   validation_split=0.2,
#   subset="training",
#   seed=123,
#   image_size=(img_height, img_width),
#   batch_size=batch_size)
#
# data_dir = 'dataset_lettuce'
# val_ds = tf.keras.utils.image_dataset_from_directory(
#   data_dir,
#   validation_split=0.2,
#   subset="validation",
#   seed=123,
#   image_size=(img_height, img_width),
#   batch_size=batch_size)
#
# class_names = train_ds.class_names
# print(class_names)

In [150]:
import pathlib
data_dir = 'dataset_lettuce/'
data_dir = pathlib.Path(data_dir)
print('directory: ',data_dir)

image_count = len(list(data_dir.glob('*/*.jpg')))
print('Number of Images: ', image_count)
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'), shuffle=False)
list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=False)

directory:  dataset_lettuce
Number of Images:  132


In [151]:
for f in list_ds:
  l = f.numpy()
  print(l)
  for i in range(len(l)):
    print(l[i])

b'dataset_lettuce\\bacterial\\ba3.jpg'
100
97
116
97
115
101
116
95
108
101
116
116
117
99
101
92
98
97
99
116
101
114
105
97
108
92
98
97
51
46
106
112
103
b'dataset_lettuce\\fungal_septoria_blight\\s05.jpg'
100
97
116
97
115
101
116
95
108
101
116
116
117
99
101
92
102
117
110
103
97
108
95
115
101
112
116
111
114
105
97
95
98
108
105
103
104
116
92
115
48
53
46
106
112
103
b'dataset_lettuce\\viral\\v6.jpg'
100
97
116
97
115
101
116
95
108
101
116
116
117
99
101
92
118
105
114
97
108
92
118
54
46
106
112
103
b'dataset_lettuce\\bacterial\\ba2.jpg'
100
97
116
97
115
101
116
95
108
101
116
116
117
99
101
92
98
97
99
116
101
114
105
97
108
92
98
97
50
46
106
112
103
b'dataset_lettuce\\fungal_powdery_mildew\\p05.jpg'
100
97
116
97
115
101
116
95
108
101
116
116
117
99
101
92
102
117
110
103
97
108
95
112
111
119
100
101
114
121
95
109
105
108
100
101
119
92
112
48
53
46
106
112
103
b'dataset_lettuce\\fungal_downy_mildew\\25.jpg'
100
97
116
97
115
101
116
95
108
101
116
116
117
99
101
92
1

In [152]:
class_names = np.array(sorted([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"]))
print(class_names)

['bacterial' 'fungal_downy_mildew' 'fungal_powdery_mildew'
 'fungal_septoria_blight' 'fungal_wilt_and_leaf_blight' 'healthy' 'viral']


In [153]:
val_size = int(image_count * 0.2)
train_ds = list_ds.skip(val_size)
val_ds = list_ds.take(val_size)

print(tf.data.experimental.cardinality(train_ds).numpy())
print(tf.data.experimental.cardinality(val_ds).numpy())

107
26


In [154]:
def get_label(file_path):
  # Convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.argmax(one_hot)

def decode_img(img):
  # Convert the compressed string to a 3D uint8 tensor
  img = tf.io.decode_jpeg(img, channels=3)
  # Resize the image to the desired size
  return tf.image.resize(img, [img_height, img_width])

def process_path(file_path):
  label = get_label(file_path)
  # Load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

In [155]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)

for image, label in train_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

Image shape:  (180, 180, 3)
Label:  2


In [156]:
def configure_for_performance(ds):
  ds = ds.cache()
  ds = ds.shuffle(buffer_size=1000)
  ds = ds.batch(batch_size)
  ds = ds.prefetch(buffer_size=AUTOTUNE)
  return ds

train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)

In [157]:
import matplotlib as plt

print(type(train_ds))
image_batch, label_batch = next(iter(train_ds))

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].numpy().astype("uint8"))
  label = label_batch[i]
  plt.title(class_names[label])
  plt.axis("off")

<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>


InvalidArgumentError: Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node DecodeJpeg}}]] [Op:IteratorGetNext]

In [ ]:
num_classes = 5

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=3
)
